### FUNCTIONS DATA CLEANING

In [1]:
import pandas as pd

from datetime import datetime

print (str(datetime.now()) )
print ()

#import imblearn
#from imblearn import under_sampling, over_sampling
#from imblearn.over_sampling import SMOTE

#path = "C:\\Users\\DOBUSER\\Documents\\RM Tim Case Study\\Complaints\\"



def CleanVioData (dfa, start_date):
    dfa['Date Received'] = pd.to_datetime(dfa['Date Received'])
    #dfa['InspectionDate'] = pd.to_datetime(dfa['InspectionDate'])
    dfa = dfa.rename(columns={'Vacate issued': 'Outcome'})


    
    #from dateutil import parser
    #start_date = parser.parse(start_date)
    #start_date = start_date.date()

    mask = (dfa['Date Received'] >= start_date)
    print ("len before date drop", len(dfa) )
    dfa = dfa.loc[mask].reset_index(drop=True)
    print ("len after date drop", len(dfa) )

    #dfa = dfa.rename(columns={'D_DATE': 'InspectionDate'})

    print ("number of nans on the vios")
    print (dfa["Outcome"].isnull().sum() )

    df2 = dfa.copy()
    df2["Count"] = 1
    
    df2 = df2[["Outcome", "Count"]]
    df2 = df2.groupby(['Outcome']).sum()
    df2 = df2.add_suffix('').reset_index()
    print (df2)
    
    percent = df2["Count"][0]/ float(df2["Count"][1])
    print ("Percent minority class:", percent)

    #df2 = df2[["Source of Complaint Description", "Count"]]
    #df2G = df2.groupby(['Source of Complaint Description']).sum()
    #df2G = df2G.add_suffix('').reset_index()
    #df2G = df2G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    #print ("percent dob" )
    #percent = df2G["Count"][1]/float(df2G["Count"].sum() )
    #print (percent)
    #print ("percent citizen")
    #percent = df2G["Count"][0]/float(df2G["Count"].sum() )
    #print (percent)
    
    return dfa


def CleanOpenData (dfb):
    print ("Length of open data", len(dfb))
    #df2 = dfb.copy()
    #print ("Drop registration id")
    #dfb = dfb.drop('RegistrationID', 1)
    #df2["Count"] = 1
    #df2 = df2[["Source of Complaint Description", "Count"]]
    #df2G = df2.groupby(['Source of Complaint Description']).sum()
    #df2G = df2G.add_suffix('').reset_index()
    #df2G = df2G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    #print ("percent dob")
    #percent = df2G["Count"][1]/float(df2G["Count"].sum() )
    #print (percent)
    #print ("percent citizen")
    #percent = df2G["Count"][0]/float(df2G["Count"].sum() )
    #print (percent)
    
    return dfb


def MergeVioOpen(dfa, dfb):
    frames = [dfa, dfb]
    df = pd.concat(frames)

    cols = df.columns

    for c in cols:
        s = c.replace(" ", "")
        #print (c)
        df = df.rename(columns={c: s})

    df["Outcome"] = df["Outcome"].fillna("Open")
    print (len(df))
    #df["SourceofComplaintDescription"] = df["SourceofComplaintDescription"].astype(str)
    #remove dob generated complaints, will bias results
    #df = df[df["SourceofComplaintDescription"] != "DOB"].reset_index(drop=True)
    #print (len(df))
    print ("rename BIN")
    df = df.rename(columns={'BINNumber': 'BIN'})
    df["BIN"] = df["BIN"].astype(str)
    df = df.reset_index(drop=True)
    return df


def CountClassesAndClean(df):
    df["Outcome"] = df["Outcome"].astype(str).map(str.strip)
    dfcount = df[df["Outcome"] == "Vacate Not Issued"]
    print ("No vios")
    print (len(dfcount) )
    dfcount = df[df["Outcome"] == "Vacate Issued"]
    print ("Vios")
    print (len(dfcount) )
    dfcount = df[df["Outcome"] == "Open"]
    print ("Open")
    print (len(dfcount) )

    return df

def DateSortAscending(df):
    df['DateReceived'] = pd.to_datetime(df['DateReceived'])
    df = df.sort_values(by = 'DateReceived', ascending=True).reset_index(drop=True)
    return df

def OutcomeConvert(f):
    outcomes = set(f["Outcome"].tolist())
    print (outcomes)
    print ("len of before drop open f", len(f))
    f = f[f["Outcome"] != "Open"].reset_index(drop=True)
    print ("len of f after drop open", len(f))
    
    f['Outcome'] = f['Outcome'].str.replace('Vacate Not Issued', '0')
    #f['Outcome'] = f['Outcome'].str.replace('Open', '0')
    f['Outcome'] = f['Outcome'].str.replace('Vacate Issued', '1')
    f['Outcome'] = f['Outcome'].astype(int)
    return f


def CleanBuildClass(df):
    print ("creating building class 2, non aggregated")
    df["DOFBuildingClassification"] = df["DOFBuildingClassification"].astype(str).map(str.strip)
    df["BldgClass"] = df["DOFBuildingClassification"].str.split(":").str[0]
    df["BldgClass"] = df["BldgClass"].str.replace('1 FAMILY DWELLINGS', '1-2_ FAMILY DWELLINGS')
    df["BldgClass"] = df["BldgClass"].str.replace('2 FAMILY DWELLINGS', '1-2_ FAMILY DWELLINGS')
    df["BldgClass"] = df["BldgClass"].str.replace('1-2_ FAMILY DWELLINGS', '1-2 FAMILY DWELLINGS')   
    df["BldgClass"] = df["BldgClass"].str.replace('MISCELLANEOUS ASYLUM, HOME', 'MISCELLANEOUS')
    df["BldgClass"] = df["BldgClass"].str.replace('MISCELLANEOUS STORE BUILDINGS', 'MISCELLANEOUS')
    df["BldgClass"] = df["BldgClass"].str.replace('MISCELLANEOUS TRANSPORTATION FACILITY', 'MISCELLANEOUS')
    df["BldgClass"] = df["BldgClass"].str.replace('MISCELLANEOUS TRANSPORTATION FACILITY', 'MISCELLANEOUS')
    df["BldgClass"] = df["BldgClass"].str.replace('TENNIS COURTS', 'SPORTS FACILITY')
    df["BldgClass"] = df["BldgClass"].str.replace('MARINAS/YACHT CLUBS', 'SPORTS FACILITY')
    df["BldgClass"] = df["BldgClass"].str.replace('MISC. OUTDOOR RECREATION FACILITY', 'SPORTS FACILITY')
    df["BldgClass"] = df["BldgClass"].str.replace('OUTDOOR POOLS', 'SPORTS FACILITY')
    df["BldgClass"] = df["BldgClass"].str.replace('PARKS RECREATION FACILITY', 'SPORTS FACILITY')
    df["BldgClass"] = df["BldgClass"].str.replace('PLAYGROUNDS', 'SPORTS FACILITY')
    df["BldgClass"] = df["BldgClass"].str.replace('STADIUMS RACE TRACKS BASEBALL FD', 'SPORTS FACILITY')
    df["BldgClass"] = df["BldgClass"].str.replace('DETENTION HSE FOR WAYWARD GIRL', 'CHILDREN RELATED')
    df["BldgClass"] = df["BldgClass"].str.replace('HOMES FOR INDIGENT KIDS/AGED/HOMELESS', 'CHILDREN RELATED')
    df["BldgClass"] = df["BldgClass"].str.replace('ORPHANAGES', 'CHILDREN RELATED')
    df["BldgClass"] = df["BldgClass"].str.replace('CHILDREN RELATEDS', 'CHILDREN RELATED')
    df["BldgClass"] = df["BldgClass"].str.replace('CHURCH, SYNAGOGUE, CHAPEL', 'RELIGIOUS')
    df["BldgClass"] = df["BldgClass"].str.replace('PARSONAGE, RECTORY', 'RELIGIOUS')
    df["BldgClass"] = df["BldgClass"].str.replace('CONVENT', 'RELIGIOUS')
    df["BldgClass"] = df["BldgClass"].str.replace('&', '_')
    df["BldgClass"] = df["BldgClass"].str.replace('/', '_')
    df["BldgClass"] = df["BldgClass"].str.replace('-', '_')
    df["BldgClass"] = df["BldgClass"].str.replace(' ', '_')
    df["BldgClass"] = df["BldgClass"].str.replace('___', '_')
    df["BldgClass"] = df["BldgClass"].str.replace('__', '_')
    pd.set_option('chained_assignment', None)
    bldClass = sorted(list(set(df["BldgClass"].tolist())))
    print (bldClass)
    
    return df

def BuildingClassAggregate(df):
    df["DOFBuildingClassification"] = df["DOFBuildingClassification"].astype(str).map(str.strip)
    df["BldgClass"] = df["DOFBuildingClassification"].str.split(":").str[0]
    df["BldgClass"] = df["BldgClass"].str.replace('1 FAMILY DWELLINGS', '1_2FAMILY_DWELLINGS')
    df["BldgClass"] = df["BldgClass"].str.replace('2 FAMILY DWELLINGS', '1_2FAMILY_DWELLINGS')
    #df["BldgClass"] = df["BldgClass"].str.replace('1 FAMILY DWELLINGS', 'Residential')
    #df["BldgClass"] = df["BldgClass"].str.replace('2 FAMILY DWELLINGS', 'Residential')
    df["BldgClass"] = df["BldgClass"].str.replace('WALK-UP APTS', 'Residential')
    df["BldgClass"] = df["BldgClass"].str.replace('ELEVATOR APTS', 'Residential')
    df["BldgClass"] = df["BldgClass"].str.replace('CONDOMINIUMS', 'Residential')
    df["BldgClass"] = df["BldgClass"].str.replace('CONDOMINIUMS', 'Residential')
    df["BldgClass"] = df["BldgClass"].str.replace('MISCELLANEOUS STORE BUILDINGS', 'Commercial')
    df["BldgClass"] = df["BldgClass"].str.replace('STORE BUILDING', 'Commercial')
    df["BldgClass"] = df["BldgClass"].str.replace('GARAGES & GAS STATIONS', 'Commercial')
    df["BldgClass"] = df["BldgClass"].str.replace('OFFICE BUILDINGS', 'Commercial')
    df["BldgClass"] = df["BldgClass"].str.replace('WAREHOUSES', 'Commercial')
    df["BldgClass"] = df["BldgClass"].str.replace('FACTORY/INDUSTRIAL BLDGS', 'Commercial')
    df["BldgClass"] = df["BldgClass"].str.replace('HOTELS', 'Commercial')
    df["BldgClass"] = df["BldgClass"].str.replace('RESIDENCE-MULTI USE', 'Multi_use')
    df["BldgClass"] = df["BldgClass"].str.replace('STORES WITH APARTMENTS ABOVE', 'Multi_use')
    df["BldgClass"] = df["BldgClass"].str.replace('SINGLE/MULT RES W/STORES/OFFICES', 'Multi_use')
    df["BldgClass"] = df["BldgClass"].str.replace('MISCELLANEOUS TRANSPORTATION FACILITY', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('MISCELLANEOUS ASYLUM, HOME', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('VACANT LAND', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('CHURCH, SYNAGOGUE, CHAPEL', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('PARSONAGE, RECTORY', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('EDUCATIONAL STRUCTURES', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('HOSPITALS & HEALTH', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('PLACES OF PUBLIC ASSEMBLY', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('DINERS FRANCHISED TYPE STAND', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('PLAYGROUNDS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('PIERS DOCKS  BULKHEADS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('SHOPPING CENTER W/PARKING FACILITY', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('PARKS RECREATION FACILITY', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('AIRPORT AIRFIELD  TERMINALS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('MISSION HOUSE NON-RESIDENTIAL', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('STADIUMS RACE TRACKS BASEBALL FD', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('OUTDOOR POOLS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('MISC. OUTDOOR RECREATION FACILITY', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('CONVENT', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('MARINAS/YACHT CLUBS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('LOFT BUILDINGS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('HOMES FOR INDIGENT KIDS/AGED/HOMELESS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('GOVERNMENT INSTALLATIONS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('FUNERAL HOME', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('DETENTION HSE FOR WAYWARD GIRLS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('DEPARTMENT STORES', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('THEATRES', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('MISCELLANEOUS', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('ORPHANAGES', 'Other')
    df["BldgClass"] = df["BldgClass"].str.replace('nan', 'Other')
    
    
    
    print ("DONE")
    return df


def convert_block(x):
    #Creates median income categorical variable
    if len(x) == 1:
        return "0000" + x
    if len(x) == 2:
        return "000" + x
    if len(x) == 3:
        return "00" + x
    if len(x) == 4:
        return "0" + x
    else:
        return x
    
def convert_lot(x):
    #Creates median income categorical variable
    if len(x) == 1:
        return "000" + x
    if len(x) == 2:
        return "00" + x
    if len(x) == 3:
        return "0" + x
    else:
        return x
  




2021-03-23 09:17:58.752173



### FUNCTIONS DATA ENGINEERING

In [2]:
print (str(datetime.now()) )
print ()

def HPDandHPDNA(df, hpd):
    hpd["BIN"] = hpd["BIN"].astype(str)
    hpdbin = hpd[["BIN"]]
    hpdbin["HPD"] = 1
    print (hpdbin["BIN"][0])
    hpdbin = hpdbin.set_index('BIN')['HPD'].to_dict()
    
    df["HPD"] = df["BIN"].map(hpdbin)
    df["HPD"] = df["HPD"].fillna(0)

    print ("sum of hpd",df["HPD"].sum() )

    print ("percent hpd", df["HPD"].sum()/len(df))

    hpdna = hpd[["BIN", "HPD Classifications"]]
    hpdna["HPD Classifications"] = hpdna["HPD Classifications"].astype(str)
    #classtypes = set(hpd["HPD Classifications"].tolist() )
    hpdna = hpdna[hpdna["HPD Classifications"] == "NOT AVAILABLE"]
    hpdna["HPD Classifications"] = 1
    hpdna["HPD Classifications"] = hpdna["HPD Classifications"].astype(int)
    hpdna = hpdna.set_index('BIN')['HPD Classifications'].to_dict()
    df["HPDNA"] = df["BIN"].map(hpdna)
    df["HPDNA"] = df["HPDNA"].fillna(0)
    
    return df

def YearBuildArea(df, dfb):
    dfb["BIN"] = dfb["BIN"].astype(str)
    yearbuilt = dfb[["BIN", "YearBuilt"]]
    yearbuilt = yearbuilt.set_index('BIN')['YearBuilt'].to_dict()
    df["YearBuilt"] = df["BIN"].map(yearbuilt)
    df["YearBuilt"] = df["YearBuilt"].fillna(0)

    buildingarea = dfb[["BIN", "BuildingArea"]]
    buildingarea = buildingarea.set_index('BIN')['BuildingArea'].to_dict()
    df["BuildingArea"] = df["BIN"].map(buildingarea)
    df["BuildingArea"] = df["BuildingArea"].fillna(0)
    
    return df

def MultipleDwellingOwner(df, dfm):
    dfm["BIN"] = dfm["BIN"].astype(str)
    dfm["BIN"] = dfm["BIN"].str.split(".").str[0]

    dfm = dfm[["BIN", "Ownership Profile", "Owner Type"]]

    cols = dfm.columns
    for c in cols:    
        s = c.replace(' ', '')
        dfm = dfm.rename(columns={c: s})
    print (cols)

    ownDic = dfm.set_index('BIN')['OwnershipProfile'].to_dict()
    df["Owner"] = df["BIN"].map(ownDic)

    #owner = df[["SWOissued", "Owner"]]
    #owner["Count"] = 1
    #owner = owner.groupby(['Owner']).sum()
    #owner = owner.add_suffix('').reset_index()
    #owner = owner.sort_values(by = 'SWOissued', ascending=False).reset_index(drop=True)
    
    #print 
    #print (owner)

    ownType = dfm.set_index('BIN')['OwnerType'].to_dict()
    df["OwnerType"] = df["BIN"].map(ownType)

    dfm["MD"] = 1
    mdDic = dfm.set_index('BIN')["MD"].to_dict()
    df["MD"] = df["BIN"].map(mdDic)
    df["MD"] = df["MD"].fillna(0)
    
    print ("len of df", len(df))
    print ("number of MDs", df["MD"].sum())
    
    return df

def CensusData(df, dc):
    dc = dc.rename(columns={"Bin": "BIN"})
    dc["BIN"] = dc["BIN"].astype(str)
    dc['MedianIncome'] = dc['MedianIncome'].fillna(0)  
    dc['MedianIncome'] = dc['MedianIncome'].astype(str)
    dc['MedianIncome'] = dc['MedianIncome'].str.split(".").str[0]
    dc['MedianIncome'] = dc['MedianIncome'].str.replace('[^\w\s]','')
    dc['MedianIncome'] = dc['MedianIncome'].astype(int)
    incomeDic = dc.set_index('BIN')['MedianIncome'].to_dict()

    df["MedianIncome"] = df["BIN"].map(incomeDic)

    print ("income null")
    print (df["MedianIncome"].isnull().sum() )

    return df


def highrise_cat(x):
    if x >= 7:
        return 1
    if x < 7:
        return 0
    
def YearBuilt(x):
    if x == 0:
        return "nan"
    if (x!=0 and x < 1900):
        return "pre_1900"
    if (x >= 1900 and x < 1916):
        return "1900_1915"
    if (x >= 1916 and x < 1938):
        return "1916_1937"   
    if (x >= 1938 and x < 1968):
        return "1938_1967"   
    if (x >= 1968 and x < 2008):
        return "1968_2007" 
    if (x >= 2008 and x < 2021):
        return "2008_2021" 
    

def BuildArea(x):
    if x == 0:
        return "nan"
    if (x!=0 and x<=3000):
        return "<3000"
    if (x>3000 and x<=6000):
        return "3000_6000"
    if (x>6000 and x<=9000):
        return "6000_9000"
    if (x>9000 and x<=12000):
        return "9000_12000"
    if (x>12000 and x<=20000):
        return "12000_20000"
    if (x>20000):
        return ">20000"
        

    
def income_cat(x):
    """Creates median income categorical variable"""
    if x == 0:
        return "NAN"
    if x > 0 and x <= 40000:
        return "Low"
    if x > 40000 and x <= 60000:
        return "Medium_Low"
    if x > 60000 and x <= 90000:
        return "Medium_High"
    if x > 90000:
        return "High"
        

""" 
def BuildArea(x):
    if x == 0:
        return "nan"
    if (x!=0 and x<=2000):
        return "<2000"
    if (x>2000 and x<=4000):
        return "2000_4000"
    if (x>4000 and x<=6000):
        return "4000_6000"
    if (x>6000 and x<=8000):
        return "6000_8000"
    if (x>8000 and x<=10000):
        return "8000_10000"
    if (x>10000 and x<=20000):
        return "10000_20000"
    if (x>20000):
        return ">20000"
        
def BuildArea(x):
    if x == 0:
        return "nan"
    if (x!=0 and x<=1000):
        return "<1000"
    if (x>1000 and x<=1500):
        return "1000_1500"
    if (x>1500 and x<=2000):
        return "1500_2000"
    if (x>2000 and x<=2500):
        return "2000_2500"
    if (x>2500 and x<=3000):
        return "2500_3000"
    if (x>3500 and x<=4000):
        return "3500_4000"    
    if (x>4000 and x<=4500):
        return "4000_4500"  
    if (x>4500 and x<=5000):
        return "4500_5000"  
    if (x>5000 and x<=10000):
        return "5000_10000"
    if (x>10000 and x<=20000):
        return "10000_20000"
    if (x>20000):
        return ">20000"

def BuildArea(x):
    if x == 0:
        return "nan"
    if (x!=0 and x<=5000):
        return "<5000"
    if (x>5000 and x<=10000):
        return "5000_10000"
    if (x>10000 and x<=15000):
        return "10000_15000"
    if (x>15000 and x<=20000):
        return "15000_20000"
    if (x>20000 and x<=25000):
        return "20000_25000"
    if (x>25000 and x<=30000):
        return "25000_30000"
    if (x>35000 and x<=40000):
        return "35000_40000"    
    if (x>40000 and x<=45000):
        return "40000_45000"  
    if (x>45000 and x<=50000):
        return "45000_50000"  
    if (x>50000 ):
        return "x>50000"
        
def BuildArea(x):
    if x == 0:
        return "nan"
    if (x!=0 and x<=10000):
        return "Small"
    if (x>10000 and x<=50000):
        return "Medium"
    if (x>50000 ):
        return "Large"
"""

2021-03-23 09:17:58.791553



' \ndef BuildArea(x):\n    if x == 0:\n        return "nan"\n    if (x!=0 and x<=2000):\n        return "<2000"\n    if (x>2000 and x<=4000):\n        return "2000_4000"\n    if (x>4000 and x<=6000):\n        return "4000_6000"\n    if (x>6000 and x<=8000):\n        return "6000_8000"\n    if (x>8000 and x<=10000):\n        return "8000_10000"\n    if (x>10000 and x<=20000):\n        return "10000_20000"\n    if (x>20000):\n        return ">20000"\n        \ndef BuildArea(x):\n    if x == 0:\n        return "nan"\n    if (x!=0 and x<=1000):\n        return "<1000"\n    if (x>1000 and x<=1500):\n        return "1000_1500"\n    if (x>1500 and x<=2000):\n        return "1500_2000"\n    if (x>2000 and x<=2500):\n        return "2000_2500"\n    if (x>2500 and x<=3000):\n        return "2500_3000"\n    if (x>3500 and x<=4000):\n        return "3500_4000"    \n    if (x>4000 and x<=4500):\n        return "4000_4500"  \n    if (x>4500 and x<=5000):\n        return "4500_5000"  \n    if (x>5000

### FUNCTIONS DUMMIES

In [3]:
print (str(datetime.now()) )
print ()


def NoPreviousJobFiling(df, dfj):
    dfj["BIN Number"] = dfj["BIN Number"].astype(str).map(str.strip)
    print ("len dfj before drop", len(dfj))
    #binList = list(set(dfj["BIN Number"].tolist()) )
    dfj = dfj.drop_duplicates(['BIN Number']).reset_index(drop=True)
    print ("len dfj after drop", len(dfj))
    
    dfj["PreviousJob"] = 0
    dfjDic = dfj.set_index('BIN Number')['PreviousJob'].to_dict()

    df["NoPreviousJob"] = df["BIN"].map(dfjDic)
    df["NoPreviousJob"] = df["NoPreviousJob"].fillna(1)
    
    print ("len of df", len(df))
    
    print ("number of complaints on BINs with no previous job filings", df["NoPreviousJob"].sum())
    print ("percent of BINs with no history of job filings", df["NoPreviousJob"].sum()/len(df))

    return df


def PreviousClass1Transactions(df, dfc, dt):
    dfc = dfc.rename(columns={'BIN-Occurrence': 'BIN', 'Vio Issued Date': 'VioDate'})
    dfc = dfc[["BIN", "VioDate"]]
    dfc["BIN"] = dfc["BIN"].astype(str)
    dfc["BIN"] = dfc["BIN"].str.split(".").str[0]
    dfc['VioDate'] = pd.to_datetime(dfc['VioDate'])
    
    dt = dt[["RECORDED / FILED", "BBL"]]
    dt = dt.rename(columns={'RECORDED / FILED': 'FiledDate'})
    dt['FiledDate'] = pd.to_datetime(dt['FiledDate'])
    dt["BBL"] = dt["BBL"].astype(str)
    dt["BBL"] = dt["BBL"].str.split(".").str[0]
    
    df["BBL_SPATIAL_KEY"] = df["BBL_SPATIAL_KEY"].astype(str)
    df["BBL_SPATIAL_KEY"] = df["BBL_SPATIAL_KEY"].str.split(".").str[0]
    
    dfbblist = list(set(df["BBL_SPATIAL_KEY"].tolist() ))
    print ("len of dt before drop", len(dt))
    dt = dt[dt['BBL'].isin(dfbblist)].reset_index(drop=True)
    print ("len of dt after drop", len(dt))

    dfbins = sorted(list(set(df["BIN"].tolist())))
    print ("len before drop", len(dfc))
    dfc = dfc[dfc['BIN'].isin(dfbins)].reset_index(drop=True)
    print ("len after drop", len(dfc))

    df["PreviousClass1"] = 0
    df["PreviousSale"] = 0
    df['DateReceived'] = pd.to_datetime(df['DateReceived'])

    thebins = df["BIN"].tolist()
    thebbls = df["BBL_SPATIAL_KEY"].tolist()
    print ("len of thebins", len(thebins))
    print ("len of df", len(df))
    datesRec = df["DateReceived"].tolist()
    
    class1VioDic = {}
    
    datethresh = 30
    
    for i in range(0, len(thebins)):
        #b = thebins[i]
        #print (b)
        dateReceived = df["DateReceived"][i]
        #print (dateReceived)
        dfc2 = dfc[dfc["BIN"] == thebins[i]]
        dt2 = dt[dt["BBL"] == thebbls[i]]
        #print ("dfc2")
        #print (dfc2)
        #print ("dt2")
        #print (thebbls[i])
        #print (dt2)
        
        if i%2000==0:
            print (i) 
        #if len(dfc2) > 0:
        mask1 = (dfc2['VioDate'] < datesRec[i])
        dfc2 = dfc2.loc[mask1]
        
        mask2 = (dt2['FiledDate'] < datesRec[i])
        dt2 = dt2.loc[mask2]
        
        #print ("after mask filter")
        #print ("date received", datesRec[i])
        #print (dt2)
        
        if len(dt2)>0:
            df["PreviousSale"][i] = 1
        

        if len(dfc2) > 0:
            #df["PreviousClass1"][i] = 1
            #class1VioDic[thebins[i]] = 1
            #print ()
            #print (dateReceived)
            #print (dfc2)
                
            dfc2["dfDateReceived"] = datesRec[i]
            dfc2["DayDiff"] = (dfc2["dfDateReceived"] - dfc2["VioDate"] ).dt.days
            dfc2 = dfc2[dfc2["DayDiff"] > datethresh]
            #print (dfc2)
            if len(dfc2) > 0:
                df["PreviousClass1"][i] = 1 
                    
    
    return df


def PreviousVacate(df):
    df['DateReceived'] = pd.to_datetime(df['DateReceived'])
    #datesRec = df["DateReceived"].tolist()

    #print ("len before drop", len(df) )
    #df2 = df[df["Outcome"] == "Vacate Issued"].reset_index(drop=True)
    #print ("len after drop", len(df2) )
    df2 = df[["BIN", "DateReceived"]]
    
    thebins = df2["BIN"].tolist()
    
    PreviousVacateDic = {}

    #datethresh = 1095
    
    for i in range(0, len(thebins)):
        b = df["BIN"][i]
        #print (b)
        df3 = df2[df2["BIN"] == thebins[i]].reset_index(drop=True)
        df3 = df3.sort_values(by = 'DateReceived', ascending=False).reset_index(drop=True)


        #print (len(df3))
        
    
        dateReceived = df3["DateReceived"][0]
        #print (dateReceived)
        if i%2000==0:
            print (i) 
        mask = (df3['DateReceived'] < dateReceived)
        
        df3 = df3.loc[mask]
        
        if len(df3) > 0:
            PreviousVacateDic[b] = 1
        
    print ("len of previous vacate dic", len(PreviousVacateDic))   
    df["PreviousVacate"] = df["BIN"].map(PreviousVacateDic)
    df["PreviousVacate"] = df["PreviousVacate"].fillna(0)
    return df

def TextDummies(df):
    df["ComplaintReason"] = df["ComplaintReason"].astype(str).map(str.strip).map(str.lower)
    print ("remove puncuation")
    df["ComplaintReason"] = df["ComplaintReason"].str.replace('[^\w\s]','')
    print ("remove numbers")
    df["ComplaintReason"] = df["ComplaintReason"].str.replace('\d+', '')
    df["ComplaintReason"] = df["ComplaintReason"].str.replace('house', 'home')
    df["ComplaintReason"] = df["ComplaintReason"].str.replace('bsmt', 'basement')

    pd.set_option('chained_assignment', None)

    print ("create text dummies")
    
    #df["txt_home"] = 0
    #df["txt_back"] = 0
    #df["txt_garage"] = 0
    #df["txt_living"] = 0
    
    df["txt_fdny"] = 0
    df["txt_occupancy"] = 0
    df["txt_cellar"] = 0
    df["txt_attic"] = 0
    df["txt_basement"] = 0
    

    sumtext = [0]

    for i in range(0, len(df)):
        if i%6000 == 0:
            print (i)
        if "fdny" in df["ComplaintReason"][i]:
            df["txt_fdny"][i] = 1
            sumtext.append(1)
        #if ("back" in df["ComplaintReason"][i] or "rear" in df["ComplaintReason"][i]):
        #    df["txt_back"][i] = 1
        #    sumtext.append(1)
        if "occupanc" in df["ComplaintReason"][i]:
            df["txt_occupancy"][i] = 1
            sumtext.append(1)
        if "cellar" in df["ComplaintReason"][i]:
            df["txt_cellar"][i] = 1
            sumtext.append(1)
        if "attic" in df["ComplaintReason"][i]:
            df["txt_attic"][i] = 1
            sumtext.append(1)
        if "basement" in df["ComplaintReason"][i]:
            df["txt_basement"][i] = 1
            sumtext.append(1)
 
    return df


def OtherDummies(df):
    #year built, building area, median income, building class, owner type
    print ("Year built")
    dummies = pd.get_dummies(df.YearBuiltCat, prefix='YB')
    dummies = dummies.drop('YB_nan', 1)
    df = pd.concat([df, dummies], axis=1)

    print ("Building area")
    dummies = pd.get_dummies(df.BuildingAreaCat, prefix='BA')
    dummies = dummies.drop('BA_nan', 1)
    df = pd.concat([df, dummies], axis=1)

    print ("Median income")
    dummies = pd.get_dummies(df.MedianIncomeCategory, prefix='MedIncome')
    dummies = dummies.drop('MedIncome_NAN', 1)
    df = pd.concat([df, dummies], axis=1)
    
    print ("Building class")
    dummies = pd.get_dummies(df.BldgClass, prefix='BC')
    #dummies = dummies.drop('BC_Other', 1)
    df = pd.concat([df, dummies], axis=1)

    print ("Owner type")
    dummies = pd.get_dummies(df.OwnerType, prefix='OT')
    df = pd.concat([df, dummies], axis=1)
    
    return df
    
    
def CommBoardDummies(df):
    df["CommunityBoard"] = df["CommunityBoard"].astype(str)
    df["CommunityBoard"] = df["CommunityBoard"].str.split(".").str[0]
    dummies = pd.get_dummies(df.CommunityBoard, prefix='CB')
    dummies = dummies.drop('CB_nan', 1)

    print ("only keep cbs 10 or greater")

    cols = dummies.columns
    dummyList = []
    #remove columns in the dummies dataframe that are less then 10, 
    #need to reduce the number of dummies so model is not overfitted
    for c in cols:
        #print (dummies[c].sum() )

        if dummies[c].sum() > 10:
            #dummies = dummies.drop(c, 1)
            dummyList.append(c)

    dummies = dummies[dummyList]
    df = pd.concat([df, dummies], axis=1)
    
    return df

def OwnerDummies(df):
    dummies = pd.get_dummies(df.Owner, prefix='O')
    print ("len of dummies", len(dummies))

    cols = dummies.columns
    dummyList = []
    #remove columns in the dummies dataframe that are less then 10, 
    #need to reduce the number of dummies so model is not overfitted
    for c in cols:
        #print (dummies[c].sum() )

        if dummies[c].sum() > 10:
            #dummies = dummies.drop(c, 1)
            dummyList.append(c)

    #remove garbage
    #dummyList.remove("O_#_#")
    #final list of owner dummies
    dummiesown = dummies[dummyList]
    df = pd.concat([df, dummiesown], axis=1)
    
    return df




"""

def PreviousClass1TransactionsTaxLiens(df, dfc, dt, dx):
    dfc = dfc.rename(columns={'BIN-Occurrence': 'BIN', 'Vio Issued Date': 'VioDate'})
    dfc = dfc[["BIN", "VioDate"]]
    dfc["BIN"] = dfc["BIN"].astype(str)
    dfc["BIN"] = dfc["BIN"].str.split(".").str[0]
    dfc['VioDate'] = pd.to_datetime(dfc['VioDate'])
    
    dt = dt[["RECORDED / FILED", "BBL"]]
    dt = dt.rename(columns={'RECORDED / FILED': 'FiledDate'})
    dt['FiledDate'] = pd.to_datetime(dt['FiledDate'])
    dt["BBL"] = dt["BBL"].astype(str)
    dt["BBL"] = dt["BBL"].str.split(".").str[0]
    
    print ("Create bbl for dx")
    dx["Block "] = dx["Block "].astype(str).map(str.strip)
    dx["Lot"] = dx["Lot"].astype(str).map(str.strip)
    dx["Borough"] = dx["Borough"].astype(str).map(str.strip)
    dx['Block '] = dx['Block '].str.split(".").str[0]
    dx['Lot'] = dx['Lot'].str.split(".").str[0]
    dx['Borough'] = dx['Borough'].str.split(".").str[0]
    dx['Block '] = dx['Block '].apply(convert_block)
    dx['Lot'] = dx['Lot'].apply(convert_lot)
    dx["BBL"] = dx["Borough"] + dx["Block "] + dx["Lot"]
    dx['Month'] = dx['Month'].str.replace('/17/', '/1/')
    dx['Month'] = pd.to_datetime(dx['Month'])
    dx = dx[["Month", "BBL"]]

    df["BBL_SPATIAL_KEY"] = df["BBL_SPATIAL_KEY"].astype(str)
    df["BBL_SPATIAL_KEY"] = df["BBL_SPATIAL_KEY"].str.split(".").str[0]
    
    dfbblist = list(set(df["BBL_SPATIAL_KEY"].tolist() ))
    print ("len of dt before drop", len(dt))
    dt = dt[dt['BBL'].isin(dfbblist)].reset_index(drop=True)
    print ("len of dt after drop", len(dt))
    
    print ("len of dx before drop by bbl", len(dx))
    dx = dx[dx['BBL'].isin(dfbblist)].reset_index(drop=True)
    print ("len of dx after drop", len(dx))

    dfbins = sorted(list(set(df["BIN"].tolist())))
    print ("len before drop", len(dfc))
    dfc = dfc[dfc['BIN'].isin(dfbins)].reset_index(drop=True)
    print ("len after drop", len(dfc))

    df["PreviousClass1"] = 0
    df["PreviousSale"] = 0
    df["PreviousTaxLien"] = 0
    df['DateReceived'] = pd.to_datetime(df['DateReceived'])

    thebins = df["BIN"].tolist()
    thebbls = df["BBL_SPATIAL_KEY"].tolist()
    print ("len of thebins", len(thebins))
    print ("len of df", len(df))
    datesRec = df["DateReceived"].tolist()
    
    class1VioDic = {}
    
    datethresh = 30
    
    #for i in range(0, len(thebins)):
    for i in range(40, 50):
        #b = thebins[i]
        #print (b)
        dateReceived = df["DateReceived"][i]
        #print (dateReceived)
        dfc2 = dfc[dfc["BIN"] == thebins[i]]
        dt2 = dt[dt["BBL"] == thebbls[i]]
        dx2 = dx[dx["BBL"] == thebbls[i]]

        #print ("dfc2")
        #print (dfc2)
        #print ("dt2")
        
        if len(dx2)>0:
            print (i)
            print (thebbls[i])
            print (dx2)
            print ()
        
        if i%2000==0:
            print (i) 
        #if len(dfc2) > 0:
        mask1 = (dfc2['VioDate'] < datesRec[i])
        dfc2 = dfc2.loc[mask1]
        
        mask2 = (dt2['FiledDate'] < datesRec[i])
        dt2 = dt2.loc[mask2]
        
        mask3 = (dx2['Month'] < datesRec[i])
        dx2 = dx2.loc[mask3]
        
        print ("after mask filter")
        print ("date received", datesRec[i])
        print (dx2)
        
        if len(dt2)>0:
            df["PreviousSale"][i] = 1
        

        if len(dfc2) > 0:
            #df["PreviousClass1"][i] = 1
            #class1VioDic[thebins[i]] = 1
            #print ()
            #print (dateReceived)
            #print (dfc2)
                
            dfc2["dfDateReceived"] = datesRec[i]
            dfc2["DayDiff"] = (dfc2["dfDateReceived"] - dfc2["VioDate"] ).dt.days
            dfc2 = dfc2[dfc2["DayDiff"] > datethresh]
            #print (dfc2)
            if len(dfc2) > 0:
                df["PreviousClass1"][i] = 1 
                    
    
    return df


def convert_block(x):
    #Creates median income categorical variable
    if len(x) == 1:
        return "0000" + x
    if len(x) == 2:
        return "000" + x
    if len(x) == 3:
        return "00" + x
    if len(x) == 4:
        return "0" + x
    else:
        return x
    
def convert_lot(x):
    #Creates median income categorical variable
    if len(x) == 1:
        return "000" + x
    if len(x) == 2:
        return "00" + x
    if len(x) == 3:
        return "0" + x
    else:
        return x
  


print ("Previous class 1 and deed trasactions dummies")
dfc  = pd.read_csv(path + "Class1_issued.csv",encoding='cp1252', low_memory=False)
dt = pd.read_csv(path + "TRANSACTIONS 11 YEARS edited.csv",encoding='cp1252', low_memory=False)

dx = pd.read_csv(path + "Tax_Lien_Sale_Lists.csv",encoding='cp1252', low_memory=False)

df = PreviousClass1TransactionsTaxLiens(df, dfc, dt, dx)



def PreviousComplaint(df, dfc):
    dfc = dfc.rename(columns={'BIN Number': 'BIN', 'D_DATE': 'DateReceived'})
    dfc["BIN"] = dfc["BIN"].astype(str)
    
    dfbins = sorted(list(set(df["BIN"].tolist())))
    
    print ("len of complaints before drop", len(dfc) )
    print ("drop based on bins in df")
    dfc = dfc[dfc['BIN'].isin(dfbins)].reset_index(drop=True)
    print ("keep only illegal conversion complaints(45)")
    dfc["Complaint  Category"] = dfc["Complaint  Category"].astype(str).map(str.strip)
    dfc = dfc[dfc["Complaint  Category"] == '45']
    print ("len of complaints after drop", len(dfc) )
    dfc = dfc[["BIN", "DateReceived"]]
    
    df["Previous45Complaint"] = 0
    
    df['DateReceived'] = pd.to_datetime(df['DateReceived'])

    thebins = df["BIN"].tolist()
    datesRec = df["DateReceived"].tolist()
    
    dfc['DateReceived'] = pd.to_datetime(dfc['DateReceived'])
    
    datethresh = 1095

    for i in range(0, len(df)):
        b = df["BIN"][i]
        #print (b)
        dateReceived = df["DateReceived"][i]
        #print (dateReceived)
        dfc2 = dfc[dfc["BIN"] == thebins[i]]
        #print (dfc2)
        if i%2000==0:
            print (i) 
        if len(dfc2) > 0:
            mask = (dfc2['DateReceived'] < datesRec[i])
            dfc2 = dfc2.loc[mask]
            #print ()
            #print (dfc2)
            if len(dfc2) > 0:
                dfc2["dfDateReceived"] = datesRec[i]
                dfc2["DayDiff"] = (dfc2["dfDateReceived"] - dfc2["DateReceived"] ).dt.days
                #print (dv2)
                dfc2 = dfc2[dfc2["DayDiff"] < datethresh]
                #print (dfc2)
                if len(dfc2) > 0:
                    df["Previous45Complaint"][i] = 1 
    return df

print ()
print ("Create previous complaint dummy")
dfc1 = pd.read_csv(path + "Complaints Building Census 2007-2010.csv",encoding='cp1252', low_memory=False)
dfc2 = pd.read_csv(path + "Complaints Building Census 2011-2015.csv",encoding='cp1252', low_memory=False)
dfc3 = pd.read_csv(path + "Complaints Building Census 2016-Present.csv",encoding='cp1252',low_memory=False )
frames = [dfc1, dfc2, dfc3]

dfc = pd.concat(frames)
dfc = dfc.reset_index(drop=True)

dfc = PreviousComplaint(df, dfc)




def PreviousClass1(df, dfc):
    dfc = dfc.rename(columns={'BIN-Occurrence': 'BIN', 'Vio Issued Date': 'VioDate'})
    dfc = dfc[["BIN", "VioDate"]]
    dfc["BIN"] = dfc["BIN"].astype(str)
    dfc["BIN"] = dfc["BIN"].str.split(".").str[0]
    dfc['VioDate'] = pd.to_datetime(dfc['VioDate'])

    dfbins = sorted(list(set(df["BIN"].tolist())))
    print ("len before drop", len(dfc))
    dfc = dfc[dfc['BIN'].isin(dfbins)].reset_index(drop=True)
    print ("len after drop", len(dfc))
    
    dfcList = dfc.values.tolist()


    df["PreviousClass1"] = 0
    df['DateReceived'] = pd.to_datetime(df['DateReceived'])

    thebins = df["BIN"].tolist()
    print ("len of thebins", len(thebins))
    print ("len of df", len(df))
    datesRec = df["DateReceived"].tolist()
    
    class1VioDic = {}
    
    #datethresh = 1095
 
    for i in range(0, len(thebins)):
        #b = thebins[i]
        #print (b)
        dateReceived = df["DateReceived"][i]
        #print (dateReceived)
        dfc2 = dfc[dfc["BIN"] == thebins[i]]
        #print (dfc2)
        if i%2000==0:
            print (i) 
        #if len(dfc2) > 0:
        mask = (dfc2['VioDate'] < datesRec[i])
        dfc2 = dfc2.loc[mask]
        if len(dfc2) > 0:
            df["PreviousClass1"][i] = 1
            #class1VioDic[thebins[i]] = 1
                #print ()
                #print (dateReceived)
                #print (dfc2)
                
                #dfc2["dfDateReceived"] = datesRec[i]
                #dfc2["DayDiff"] = (dfc2["dfDateReceived"] - dfc2["DateReceived"] ).dt.days
                #print (dv2)
                #dfc2 = dfc2[dfc2["DayDiff"] < datethresh]
                #print (dfc2)
                #if len(dfc2) > 0:
                #    df["Previous45Complaint"][i] = 1 
                    

    return lol

test = PreviousClass1(df, dfc)



def PreviousVacate(df):
    
    from operator import itemgetter

    
    df['DateReceived'] = pd.to_datetime(df['DateReceived'])
    #datesRec = df["DateReceived"].tolist()

    #print ("len before drop", len(df) )
    #df2 = df[df["Outcome"] == "Vacate Issued"].reset_index(drop=True)
    #print ("len after drop", len(df2) )
    df2 = df[["BIN", "DateReceived"]]
    df2["BIN"] = df2["BIN"].astype(int)
    thebins = df2["BIN"].tolist()

    df2 = df2.values.tolist()

    
    
    PreviousVacateDic = {}

    #datethresh = 1095
    
    newList = []
    
    for i in range(0, len(df2)):
        #b = df["BIN"][i]
        #print (b)
        
        b = df2[i][0]
        
        for j in range(0, len(df2)):
            if df2[j][0] == b:
                #print (df2[j][0], df2[j][1])
                newList.append([df2[j][0], df2[j][1]])
        
        
        newList = sorted(newList, key=itemgetter(1), reverse=True)
        recentdateReceived = newList[0][1]
        
        newList2 = []
        for j in range(0, len(newList)):
            if newList[j][1] < recentdateReceived:
                newList2.append([newList[j][0], newList[j][1]])


        #print (b, " ", recentdateReceived)
        #print (newList)
        #print (newList2)
        
        if i%2000==0:
            print (i) 
       
        if len(newList2) > 0:
            PreviousVacateDic[b] = 1
        
        newList = []
        newList2 = []
        
    print ("len of previous vacate dic", len(PreviousVacateDic))   
    df["PreviousVacate"] = df["BIN"].map(PreviousVacateDic)
    df["PreviousVacate"] = df["PreviousVacate"].fillna(0)
  
   
    return df

dt = PreviousVacate(df)
"""




2021-03-23 09:17:59.356259



'\n\ndef PreviousClass1TransactionsTaxLiens(df, dfc, dt, dx):\n    dfc = dfc.rename(columns={\'BIN-Occurrence\': \'BIN\', \'Vio Issued Date\': \'VioDate\'})\n    dfc = dfc[["BIN", "VioDate"]]\n    dfc["BIN"] = dfc["BIN"].astype(str)\n    dfc["BIN"] = dfc["BIN"].str.split(".").str[0]\n    dfc[\'VioDate\'] = pd.to_datetime(dfc[\'VioDate\'])\n    \n    dt = dt[["RECORDED / FILED", "BBL"]]\n    dt = dt.rename(columns={\'RECORDED / FILED\': \'FiledDate\'})\n    dt[\'FiledDate\'] = pd.to_datetime(dt[\'FiledDate\'])\n    dt["BBL"] = dt["BBL"].astype(str)\n    dt["BBL"] = dt["BBL"].str.split(".").str[0]\n    \n    print ("Create bbl for dx")\n    dx["Block "] = dx["Block "].astype(str).map(str.strip)\n    dx["Lot"] = dx["Lot"].astype(str).map(str.strip)\n    dx["Borough"] = dx["Borough"].astype(str).map(str.strip)\n    dx[\'Block \'] = dx[\'Block \'].str.split(".").str[0]\n    dx[\'Lot\'] = dx[\'Lot\'].str.split(".").str[0]\n    dx[\'Borough\'] = dx[\'Borough\'].str.split(".").str[0]\n    dx[\

### SAMPLING

In [4]:
print (str(datetime.now()) )
print ()

def RandomUpsampleTraining(df2):
    print ("create open complaint dataset and then train, validation and test datasets")

    opendf = df2[df2["Outcome"] == "Open"]
    print ("len of opendf", len(opendf))

    df2 = df2[df2["Outcome"] != "Open"].reset_index(drop=True)
    print ("len of df2", len(df2))
    
    #convert swo issued/not issued to 0/1
    df2 = OutcomeConvert(df2)
    
    df2["Index"] = df2.index


    def BalanceClass(df2):
        # Separate majority and minority classes
        df_majority = df2[df2.Outcome==0]
        df_minority = df2[df2.Outcome==1]

        print ("Size of majority class", len(df_majority) )
        print ("Size of minority class", len(df_minority) )

        lenMajority = len(df_majority)


        # Upsample minority class
        df_minority_upsampled = resample(df_minority, 
                                         replace=True,     # sample with replacement
                                         n_samples=lenMajority, # to match majority class
                                         random_state=123) # reproducible results

        print ("Size of minority class upsampled", len(df_minority_upsampled) )


        # Combine majority class with upsampled minority class
        df2 = pd.concat([df_majority, df_minority_upsampled])
        df2 = df2.reset_index(drop=True)

        df2 = df2.fillna(0)
        print ("Size of final dataset ", len(df2) )


        return df2


    print ("Get holdout data that is not upsampled")

    print ("Randomly sort df2" )
    #print ("NOT RANDOMLY SORTING BEFORE UPSAMPLE")
    #df2bal["Index"] = df2bal.index
    df2 = df2.sample(frac=1)
    df2 = df2.reset_index(drop=True)

    lengthdf2 = int(len(df2))
    lengthdf2_20perc = int(lengthdf2*0.20)
    lengthdf2_60perc = int(lengthdf2*0.60)

    #get non-upsampled 
    print ("get the imbalanced holdout and test")
    df2hold = df2[lengthdf2_60perc + lengthdf2_20perc:]
    print ("df2hold", len(df2hold))
    
    df2testImbal = df2[lengthdf2_60perc:lengthdf2_60perc + lengthdf2_20perc]
    print ("df2testImbal", len(df2testImbal))

    print ("Get the training then upsample training only")
    #df3 = df2[:lengthdf2_60perc + lengthdf2_20perc]
    df3 = df2[:lengthdf2_60perc]

    print ("Balance the class")
    df2bal = BalanceClass(df3)

    print ("Randomly sort df2bal" )
    #df2bal["Index"] = df2bal.index
    df2bal = df2bal.sample(frac=1)
    df2bal = df2bal.reset_index(drop=True)

    #lengthdf2 = int(len(df2bal))
    #lengthdf2_20perc = int(lengthdf2*0.20)
    #lengthdf2_60perc = int(lengthdf2*0.60)

    #print ("lengthdf2bal", lengthdf2)
    #print ("lengthdf2bal_60perc",  lengthdf2_60perc)
    #print ( "lengthdf2bal_20perc", lengthdf2_20perc )


    #df2trainBal = df2bal[0:lengthdf2_60perc]
    df2trainBal = df2bal.copy()
    print ("df2trainBal", len(df2trainBal))
    

    #df2holdBal = df2bal[lengthdf2_60perc + lengthdf2_20perc:]
    #print ("df2holdBal", len(df2holdBal))
    
    return df2trainBal, df2testImbal, df2hold, opendf



def RandomUpsampleTrainingTest(df2):
    print ("create open complaint dataset and then train, validation and test datasets")

    opendf = df2[df2["Outcome"] == "Open"]
    print ("len of opendf", len(opendf))

    df2 = df2[df2["Outcome"] != "Open"].reset_index(drop=True)
    print ("len of df2", len(df2))
    
    #convert swo issued/not issued to 0/1
    df2 = OutcomeConvert(df2)
    
    df2["Index"] = df2.index


    def BalanceClass(df2):
        # Separate majority and minority classes
        df_majority = df2[df2.Outcome==0]
        df_minority = df2[df2.Outcome==1]

        print ("Size of majority class", len(df_majority) )
        print ("Size of minority class", len(df_minority) )

        lenMajority = len(df_majority)


        # Upsample minority class
        df_minority_upsampled = resample(df_minority, 
                                         replace=True,     # sample with replacement
                                         n_samples=lenMajority, # to match majority class
                                         random_state=123) # reproducible results

        print ("Size of minority class upsampled", len(df_minority_upsampled) )


        # Combine majority class with upsampled minority class
        df2 = pd.concat([df_majority, df_minority_upsampled])
        df2 = df2.reset_index(drop=True)

        df2 = df2.fillna(0)
        print ("Size of final dataset ", len(df2) )


        return df2


    print ("Get holdout data that is not upsampled")

    print ("Randomly sort df2" )
    #df2bal["Index"] = df2bal.index
    df2 = df2.sample(frac=1)
    df2 = df2.reset_index(drop=True)

    lengthdf2 = int(len(df2))
    lengthdf2_20perc = int(lengthdf2*0.20)
    lengthdf2_60perc = int(lengthdf2*0.60)

    #get non-upsampled 
    df2hold = df2[lengthdf2_60perc + lengthdf2_20perc:]
    print ("df2hold", len(df2hold))

    print ("Get the training and test dataset and then upsample")
    df3 = df2[:lengthdf2_60perc + lengthdf2_20perc]


    print ("Balance the class")
    df2bal = BalanceClass(df3)

    print ("Randomly sort df2bal" )
    #df2bal["Index"] = df2bal.index
    df2bal = df2bal.sample(frac=1)
    df2bal = df2bal.reset_index(drop=True)

    lengthdf2 = int(len(df2bal))
    lengthdf2_20perc = int(lengthdf2*0.20)
    lengthdf2_60perc = int(lengthdf2*0.60)

    print ("lengthdf2bal", lengthdf2)
    print ("lengthdf2bal_60perc",  lengthdf2_60perc)
    print ( "lengthdf2bal_20perc", lengthdf2_20perc )


    df2trainBal = df2bal[0:lengthdf2_60perc]
    print ("df2trainBal", len(df2trainBal))
    df2testBal = df2bal[lengthdf2_60perc:lengthdf2_60perc + lengthdf2_20perc]
    print ("df2testBal", len(df2testBal))
    #df2holdBal = df2bal[lengthdf2_60perc + lengthdf2_20perc:]
    #print ("df2holdBal", len(df2holdBal))
    
    return df2trainBal, df2testBal, df2hold


def DropFields(f):
    f = f.drop('BuildingArea', 1)
    f = f.drop('YearBuilt', 1)
    f = f.drop('P_NUMBER_OF_STORIES', 1)
    f = f.drop('MedianIncome', 1)
    f = f.drop('YearBuiltCat', 1)
    f = f.drop('BuildingAreaCat', 1)
    f = f.drop('MedianIncomeCategory', 1)
    f = f.drop('OwnerType', 1)
    f = f.drop('BldgClass', 1)
    f = f.drop('CommunityBoard', 1)
    f = f.drop('Owner', 1)
    f = f.drop('DateReceived', 1)
    f = f.drop('ComplaintCategory', 1)
    f = f.drop('ComplaintCategoryDescription', 1)
    f = f.drop('ComplaintNumber', 1)
    f = f.drop('BIN', 1)
    f = f.drop('BoroughName', 1)
    f = f.drop('Latitude', 1)
    f = f.drop('Longitude', 1)
    f = f.drop('DOFBuildingClassification', 1)
    f = f.drop('FinanceOwnerName', 1)
    f = f.drop('HouseNumber', 1)
    f = f.drop('StreetName', 1)
    f = f.drop('RegistrationID', 1)
    f = f.drop('Index', 1)
    f = f.drop('SourceofComplaintDescription', 1)
    f = f.drop('ComplaintReason', 1)
    f = f.drop('BBL_SPATIAL_KEY', 1)

     

    return f

2021-03-23 09:18:03.013567



### FUNCTION CALLS DATA PREPERATION 

In [5]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from sklearn.utils import resample
pd.set_option('chained_assignment', None)


path = "C:\\Users\\DOBUSER\\Documents\\RM Tim Case Study\\Illegal Conversions\\"

print ("Read violations data")
start_date = "1-1-2010"
print ("Clean vio data")
dfa = pd.read_csv(path + "RapidMiner 45_violations.csv",encoding='cp1252') # encoding='cp1252'
dfa = CleanVioData (dfa, start_date)

print ()
print ("Read open data")
dfb = pd.read_csv(path + "RapidMiner 45_open_complaints.csv",encoding='cp1252' )
dfb = CleanOpenData(dfb)

print ()
print ("Merge vio and open data")
df = MergeVioOpen(dfa, dfb)

print ()
df = CountClassesAndClean(df)

#print ("Sort date ascending")
#df = DateSortAscending(df)

print ("Non aggregated building class")
df = CleanBuildClass(df)

print ()
#print ("Aggregated building class")
#df = BuildingClassAggregate(df)


#print ()
#print ("Insert class 1 and wwp vios")
#dv = pd.read_csv(path + "Class1_issued.csv", encoding='cp1252')
#dw = pd.read_csv(path + "RapidMiner wwp_vios.csv", encoding='cp1252')
#df = Class1WWPVios(df, dv, dw)

#print ("Cluster sample")
#df = ClusterSample(df)

print ()
print ("Insert hpd and hpd NA")
hpd = pd.read_csv(path + "Comprehensive 1- 317K HPD List.csv", encoding='cp1252')
df = HPDandHPDNA(df, hpd)

print ()
print ("Insert year and building area")
dfb = pd.read_csv(path + "Pluto Property Info by BIN.csv", encoding='cp1252')
dfb = dfb.rename(columns={'P_BIN': 'BIN', 'Year Built': 'YearBuilt', 'Building Area':'BuildingArea'})
df = YearBuildArea(df, dfb)


print ()
print ("Insert multple dwelling owner")
pathmd = "C:\\Users\\DOBUSER\\Documents\\DATA\\Ownership\\"
dfm = pd.read_csv(pathmd + "Multiple Dwelling Ownership Profile Single Mode ALL.csv")
df = MultipleDwellingOwner(df, dfm)

print ()
print ("Insert census data")
dc = pd.read_csv(path + "Census Income data.csv")
df = CensusData(df, dc)

print ()
print ("CREATE CATEGORICAL VARIABLES")
print("Create high rise category")  
df["HighRise"] = df['P_NUMBER_OF_STORIES'].apply(highrise_cat)

print ("Create year built category")
df["YearBuiltCat"] = df['YearBuilt'].apply(YearBuilt)

print ("Create Building area Category")
df["BuildingAreaCat"] = df['BuildingArea'].apply(BuildArea)

print ("Create income category")
df['MedianIncome'] = df['MedianIncome'].fillna(0)    
df['MedianIncomeCategory'] = df['MedianIncome'].apply(income_cat)

print ()
print ("CREATE DUMMIES")
print ("No previous job filings")
dfj  = pd.read_csv(path + "Job filling unique_bin.csv",encoding='cp1252', low_memory=False)
df = NoPreviousJobFiling(df, dfj)
print ("Previous class 1 and deed trasactions dummies")
dfc  = pd.read_csv(path + "Class1_issued.csv",encoding='cp1252', low_memory=False)
dt = pd.read_csv(path + "TRANSACTIONS 11 YEARS edited.csv",encoding='cp1252', low_memory=False)
df = PreviousClass1Transactions(df, dfc, dt)
print ("Create previous vacate dummy")
df = PreviousVacate(df)

print ("Create text dummies")
df = TextDummies(df)



print ("Create year built, building area, median income, build class and owner type dummies")
df = OtherDummies(df)
df = df.reset_index(drop=True)

print ("Create community board dummies")
df = CommBoardDummies(df)
print ("Shape of final df", df.shape)
#print ("Create owner dummies")
#df = OwnerDummies(df)

print ()
print ('SAMPLING')
#print ("Random upsample, training and test")
#df2trainBal, df2testBal, df2hold = RandomUpsampleTrainingTest(df)
print ("Random upsample, training only")
df2trainBal, df2testImbal, df2hold, opendf = RandomUpsampleTraining(df)


print ()
print ("DROP FIELDS")
print ("Training")
df2trainBal = DropFields(df2trainBal)
print ("Training shape",df2trainBal.shape )
print ("Testing")
#df2testBal = DropFields(df2testBal)
df2testImbal = DropFields(df2testImbal)
print ("Testing shape", df2testImbal.shape)





[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DOBUSER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Read violations data
Clean vio data
len before date drop 54377
len after date drop 52685
number of nans on the vios
0
             Outcome  Count
0      Vacate Issued   3890
1  Vacate Not Issued  48795
Percent minority class: 0.07972128291833179

Read open data
Length of open data 4089

Merge vio and open data
56774
rename BIN

No vios
48795
Vios
3890
Open
4089
Non aggregated building class
creating building class 2, non aggregated
['1_2_FAMILY_DWELLINGS', 'CHILDREN_RELATED', 'CONDOMINIUMS', 'DEPARTMENT_STORES', 'DINERS_FRANCHISED_TYPE_STAND', 'EDUCATIONAL_STRUCTURES', 'ELEVATOR_APTS', 'FACTORY_INDUSTRIAL_BLDGS', 'FUNERAL_HOME', 'GARAGES_GAS_STATIONS', 'GOVERNMENT_INSTALLATIONS', 'HOSPITALS_HEALTH', 'HOTELS', 'MISCELLANEOUS', 'MISSION_HOUSE_NON_RESIDENTIAL', 'OFFICE_BUILDINGS', 'PIERS_DOCKS_BULKHEADS', 'PLACES_OF_PUBLIC_ASSEMBLY', 'RELIGIOUS', 'RESIDENCE_MULTI_USE', 'SHOPPING_CENTER_W_PARKING_FACILITY', 'SINGLE_MULT_RES_W_STORES_OFFICES', 'SPORTS_FACILITY', 'STORES_WITH_APARTMENTS_ABOV

C:\Users\DOBUSER\anaconda3_8\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['BIN', 'Ownership Profile', 'Owner Type'], dtype='object')
len of df 56774
number of MDs 21787.0

Insert census data
income null
2709

CREATE CATEGORICAL VARIABLES
Create high rise category
Create year built category
Create Building area Category
Create income category

CREATE DUMMIES
No previous job filings
len dfj before drop 416583
len dfj after drop 416583
len of df 56774
number of complaints on BINs with no previous job filings 18933.0
percent of BINs with no history of job filings 0.3334801141367527
Previous class 1 and deed trasactions dummies
len of dt before drop 673447
len of dt after drop 34243
len before drop 316954
len after drop 84677
len of thebins 56774
len of df 56774
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
Create previous vacate dummy
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
3600

In [14]:
df.to_csv(path + "Illegal Conversions Violations Data.csv", index=False)

In [6]:
df.dtypes

DateReceived                    datetime64[ns]
ComplaintCategory                        int64
ComplaintCategoryDescription            object
ComplaintNumber                          int64
BIN                                     object
                                     ...      
CB_413                                   uint8
CB_414                                   uint8
CB_501                                   uint8
CB_502                                   uint8
CB_503                                   uint8
Length: 151, dtype: object

### COUNT CATEGORIES

In [ ]:
print("Aggregated Building Class")
pd.set_option('chained_assignment', None)
bldClass = sorted(list(set(df["BldgClass"].tolist())))
#print (bldClass)               
bldclass = df[["BldgClass", "Outcome"]]  
bldclass = OutcomeConvert(bldclass)
bldclass["Count"] = 1
bldclass = bldclass.groupby(['BldgClass']).sum()
bldclass = bldclass.add_suffix('').reset_index()
bldclass['Count'] = bldclass['Count'].astype(float)
bldclass["Percent"] = bldclass['Outcome']/bldclass['Count']
#bldclass["Percent"] = bldclass['SWOissued']/bldclass['Count'].sum()
bldclass["Percent"] = bldclass["Percent"]*100
#sumperc = bldclass["Percent"].sum()
#print (sumperc)
bldclass = bldclass.sort_values(by = 'Outcome', ascending=False).reset_index(drop=True)
bldclass["SWOPercentTotal"] = (bldclass["Outcome"]/float(bldclass["Outcome"].sum()))*100
print (bldclass)

print ()
yearbuilt = df[["Outcome", "YearBuiltCat"]]
yearbuilt = OutcomeConvert(yearbuilt)
yearbuilt["Count"] = 1
yearbuilt = yearbuilt.groupby(['YearBuiltCat']).sum()
yearbuilt = yearbuilt.add_suffix('').reset_index()
yearbuilt['Count'] = yearbuilt['Count'].astype(float)
print (yearbuilt)

buildingarea = df[["Outcome", "BuildingAreaCat"]]
buildingarea = OutcomeConvert(buildingarea)
buildingarea["Count"] = 1
buildingarea = buildingarea.groupby(['BuildingAreaCat']).sum()
buildingarea = buildingarea.add_suffix('').reset_index()
buildingarea['Count'] = buildingarea['Count'].astype(float)
print (buildingarea)

print ()
highrise = df[["Outcome", "HighRise"]]
highrise = OutcomeConvert(highrise)
highrise["Count"] = 1
highrise = highrise.groupby(['HighRise']).sum()
highrise = highrise.add_suffix('').reset_index()
highrise['Count'] = highrise['Count'].astype(float)
print (highrise)

print()
medincome = df[["Outcome", "MedianIncomeCategory"]]
medincome = OutcomeConvert(medincome)
medincome["Count"] = 1
medincome = medincome.groupby(['MedianIncomeCategory']).sum()
medincome = medincome.add_suffix('').reset_index()
medincome['Count'] = medincome['Count'].astype(float)
print (medincome)


print()
owntype = df[["Outcome", "OwnerType"]]
owntype = OutcomeConvert(owntype)
owntype["Count"] = 1
owntype = owntype.groupby(['OwnerType']).sum()
owntype = owntype.add_suffix('').reset_index()
owntype = owntype.sort_values(by = 'Outcome', ascending=False).reset_index(drop=True)
print ("Owner type")
print (owntype)


print()
comboard = df[["Outcome", "CommunityBoard"]]
comboard = OutcomeConvert(comboard)
comboard["Count"] = 1
comboard = comboard.groupby(['CommunityBoard']).sum()
comboard = comboard.add_suffix('').reset_index()
#comboard['Count'] = comboard['Count'].astype(float)
comboard = comboard.sort_values(by = 'Outcome', ascending=False).reset_index(drop=True)
print (comboard[:20])


job = df[["Outcome", "NoPreviousJob"]]
job = OutcomeConvert(job)
job = job.groupby(['NoPreviousJob']).sum()
job = job.add_suffix('').reset_index()

print (job)
print (job["Outcome"]/job["Outcome"].sum())


"""

print ()
print ("Sum of SWO on CB",comboard["SWOissued"].sum() )
print ("Number of CBs", len(set(comboard["CommunityBoard"].tolist())))
print ("Sum of SWO on CB Top 10", comboard["SWOissued"][0:10].sum())
print ("Sum of SWO on CB Top 20", comboard["SWOissued"][0:20].sum())
print ("Sum of SWO on CB Top 30", comboard["SWOissued"][0:30].sum())


print("Non-Aggregated Building Class")
pd.set_option('chained_assignment', None)
bldClass = sorted(list(set(df["BldgClass2"].tolist())))
#print (bldClass)               
bldclass = df[["BldgClass2", "Outcome"]]  
bldclass = OutcomeConvert(bldclass)
bldclass["Count"] = 1
bldclass = bldclass.groupby(['BldgClass2']).sum()
bldclass = bldclass.add_suffix('').reset_index()
bldclass['Count'] = bldclass['Count'].astype(float)
bldclass["Percent"] = bldclass['Outcome']/bldclass['Count']
#bldclass["Percent"] = bldclass['SWOissued']/bldclass['Count'].sum()
bldclass["Percent"] = bldclass["Percent"]*100
#sumperc = bldclass["Percent"].sum()
#print (sumperc)
bldclass = bldclass.sort_values(by = 'Outcome', ascending=False).reset_index(drop=True)
bldclass["SWOPercentTotal"] = (bldclass["Outcome"]/float(bldclass["Outcome"].sum()))*100
#print (bldclass)


"""

In [ ]:
job["Outcome"]/job["Outcome"].sum()

### MODEL SELECTION

In [7]:
print (str(datetime.now()) )
print ()

import pandas as pd
import numpy as np
import math

    
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.model_selection import train_test_split
#from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
#create data
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_blobs
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

    
model_selection = []
    
def AccuracyAndAUC(model, clf, X_test, y_test, a, b, c):
    a = "(" + a + ","
    b = b + "," 
    c = c +  ")"
    accuracy_param = a + b + c
    clf_predicted = clf.predict(X_test)
        
    pred_prob = clf.predict_proba(X_test)[:,1]
    roc=roc_auc_score(y_test, pred_prob)
    #print ('AUC:', round(roc, 2))

        
    #clf_predicted = clf.predict(X_test)
    #confusion = confusion_matrix(y_test, clf_predicted)

    #print('Classifier Confusion Matrix\n', confusion)
    print("precision:", precision_score(y_test, clf_predicted), "recall:", recall_score(y_test, clf_predicted),accuracy_param )
        
    model_selection.append([model,accuracy_param, clf.score(X_train, y_train), accuracy_score(y_test, clf_predicted),
                               precision_score(y_test, clf_predicted),recall_score(y_test, clf_predicted), 
                                f1_score(y_test, clf_predicted), roc])
    
        
        
        
def AccuracySVM(clf, X_test, y_test, a, b):
    a = "(" + a + ","
    b = b + ")"
    clf_predicted = clf.predict(X_test)
    print ()
    print ("Accuracy on these parameters", a, b)
    print('Accuracy on test set: {:.2f}'.format(accuracy_score(y_test, clf_predicted)))
    print('Precision: {:.2f}'.format(precision_score(y_test, clf_predicted)))
    print('Recall: {:.2f}'.format(recall_score(y_test, clf_predicted)))
    print('F1: {:.2f}'.format(f1_score(y_test, clf_predicted)))
        
    #pred_prob = clf.predict_proba(X_test)[:,1]
    #roc=roc_auc_score(y_test, pred_prob)
    #print ('AUC:', round(roc, 2))
        
    y_score_gb = clf.fit(X_train, y_train).decision_function(X_test)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_gb)
    roc_auc_gb = auc(fpr_lr, tpr_lr)
    print ("AUC:", round(roc_auc_gb, 2))


    clf_predicted = clf.predict(X_test)
    confusion = confusion_matrix(y_test, clf_predicted)

    print('Classifier Confusion Matrix\n', confusion)
        
        
def PlotROC(clf, X_test, y_test):
    y_score_gb = clf.fit(X_train, y_train).decision_function(X_test)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_gb)
    roc_auc = auc(fpr_lr, tpr_lr)
    #PLOT
    plt.figure()
    plt.xlim([-0.01, 1.00])
    plt.ylim([-0.01, 1.01])
    plt.plot(fpr_lr, tpr_lr, lw=3, label='ROC curve (area = {:0.2f})'.format(roc_auc))
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
    plt.legend(loc='lower right', fontsize=13)
    plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
    plt.axes().set_aspect('equal')
    plt.show()
        
    return plt.show()
    

    

#RANDOM
y_train = df2trainBal["Outcome"]
X_train = df2trainBal.drop('Outcome', 1)
#print ("Reducing indep vars using feature importance")
#X_train = X_train[importantfeatures]


#y_test = df2testBal["Outcome"]
#X_test = df2testBal.drop('Outcome', 1)

y_test = df2testImbal["Outcome"]
X_test = df2testImbal.drop('Outcome', 1)
#X_test = X_test[importantfeatures]


#test baseline using dummy classifier

dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
    
print ("Baseline Accuracy: Dummy majority score:", round(dummy_majority.score(X_test, y_test), 2) )
#print ()

print ("IMPLEMENT SEVERAL MODELS" )

#hidden_layers = [(10), (20), (30), (40), (50), (60), (70), (80), (90), (100) ]

hidden_layers = [(100), (100,100),  (100,100,100)  ]

#hidden_layers = [(20),  (60),(100),
#                 (20,20), (60,60), (100,100),
#               (20,20,20),(60,60,60), (100,100,100)  ]


#hidden_layers = [(100)]

#hidden_layers = [(10), (20), (30), (40), (50), (60), (70), (80), (90), (100), (150), (200),
#                 (10,10), (20,20), (30,30), (40,40), 
#                 (50,50), (60,60), (70,70),
#                 (10,10,10), (20,20,20), (30,30,30), 
#                 (40,40,40), (50,50,50), (60,60,60), (70,70,70)]

#hidden_layers = [(20),  (40),(60), (80), (100), (120), (140), (160), (180), (200),
#                 (20,20), (40,40), (60,60), (80,80), (100,100), (120,120), (140,140), (160,160), (180, 180), (200,200),
#                (20,20,20), (40,40,40), (60,60,60), (80,80,80), (100,100,100), (120,120,120), (140,140,140), (160,160,160),
#                (180,180, 180), (200, 200, 200), (250, 250, 250),(300, 300, 300)  ]

iterations = 1000

#NEURAL NETWORKS
print ("Neural Networks")

"""
for layer in hidden_layers:
    clf = MLPClassifier(hidden_layer_sizes = layer, max_iter=iterations) 
    clf.fit(X_train, y_train)
    modelstr = "Neural Networks"
    a = str(layer)
    b = str(iterations)
    print (modelstr, a)
    AccuracyAndAUC(modelstr, clf, X_test, y_test, a, b)


#print ("Naive Bayes")
#clf = GaussianNB().fit(X_train, y_train)
#modelstr = "Naive Bayes"
#a = "default"
#b = "default"
#AccuracyAndAUC(modelstr, clf, X_test, y_test, a, b)
"""

print ("Logistic Regression")
this_C = [0.001, 0.01, 0.1, 1, 100]
modelstr = "Logistic Regression"

for the_c in this_C:
    a = "C=" + str(the_c)
    b = "default"
    c = "na"
    clf = LogisticRegression(C=the_c).fit(X_train, y_train)
    AccuracyAndAUC(modelstr, clf, X_test, y_test, a, b, c)


print ("Gradient boosted trees")
learn_rate = [0.0001, 0.001, 0.01, 0.1,0.2,0.3,0.4, 0.5]
#learn_rate = [ 0.01, 0.1,0.2,0.3,0.4, 0.5]
mdepth = [2, 3, 4, 5, 6]
#est = [100, 200, 300]
est = [100]


for learn in learn_rate:
    for depth in mdepth:
        for e in est:
            clf = GradientBoostingClassifier(learning_rate = learn, max_depth = depth,n_estimators=e, random_state = 0)
            clf.fit(X_train, y_train)
            modelstr = "GBT"
            a = "learn rate=" + str(learn)
            b = "max depth=" + str(depth)
            c = "n_estim" + str(e)
            AccuracyAndAUC(modelstr, clf, X_test, y_test, a, b, c)


print ("Random forests")

estimators = [1, 5,10,15,20, 25, 30, 40, 50]

for est in estimators:
    clf = RandomForestClassifier(n_estimators = est,random_state=0).fit(X_train, y_train)
    modelstr = "Random Forests"
    a = "Estimators=" + str(est)
    b = "default"
    c = "na"
    AccuracyAndAUC(modelstr, clf, X_test, y_test, a, b, c)
    

#clf = RandomForestClassifier(max_features = 8, random_state = 0)
    
df_model = pd.DataFrame(model_selection, columns=('Model','Parameters', 'Training Score', 'Test Score', 'Precision', 'Recall', 'F1', 'AUC'))


2021-03-23 09:53:08.665516

Baseline Accuracy: Dummy majority score: 0.93
IMPLEMENT SEVERAL MODELS
Neural Networks
Logistic Regression
precision: 0.14959178298656833 recall: 0.7217280813214739 (C=0.001,default,na)
precision: 0.15977653631284916 recall: 0.7268106734434562 (C=0.01,default,na)


C:\Users\DOBUSER\anaconda3_8\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


precision: 0.15814341300722623 recall: 0.7229987293519695 (C=0.1,default,na)


C:\Users\DOBUSER\anaconda3_8\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


precision: 0.15667311411992263 recall: 0.7204574332909784 (C=1,default,na)


C:\Users\DOBUSER\anaconda3_8\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


precision: 0.15704070407040704 recall: 0.7255400254129606 (C=100,default,na)
Gradient boosted trees
precision: 0.10294659300184163 recall: 0.7102922490470139 (learn rate=0.0001,max depth=2,n_estim100)
precision: 0.1043605181536216 recall: 0.7268106734434562 (learn rate=0.0001,max depth=3,n_estim100)
precision: 0.10514305774969282 recall: 0.761118170266836 (learn rate=0.0001,max depth=4,n_estim100)
precision: 0.12987918215613384 recall: 0.7102922490470139 (learn rate=0.0001,max depth=5,n_estim100)
precision: 0.13201992753623187 recall: 0.7407878017789072 (learn rate=0.0001,max depth=6,n_estim100)
precision: 0.10433847257081391 recall: 0.7395171537484116 (learn rate=0.001,max depth=2,n_estim100)
precision: 0.1043605181536216 recall: 0.7268106734434562 (learn rate=0.001,max depth=3,n_estim100)
precision: 0.14491017964071856 recall: 0.6149936467598475 (learn rate=0.001,max depth=4,n_estim100)
precision: 0.1297071129707113 recall: 0.7090216010165185 (learn rate=0.001,max depth=5,n_estim100)

In [8]:
df_model = df_model.sort_values(by = 'F1', ascending=False).reset_index(drop=True)
#df_model.to_csv(path + "GBT Results.csv", index=False)
#df_model2 = df_model[df_model["Recall"] > 0.7]
#df_model2 = df_model2[df_model2["F1"] > 0.25]

#df_model2["DiffTrainTest"] = df_model2["Training Score"] - df_model2["Test Score"]
#df_model2 = df_model2.sort_values(by = 'DiffTrainTest', ascending=True).reset_index(drop=True)

df_model

,Model,Parameters,Training Score,Test Score,Precision,Recall,F1,AUC
0,GBT,"(learn rate=0.3,max depth=2,n_estim100)",0.713065,0.700104,0.161484,0.719187,0.263747,0.768951
1,Logistic Regression,"(C=0.01,default,na)",0.701259,0.694125,0.159777,0.726811,0.261965,0.772806
2,GBT,"(learn rate=0.4,max depth=2,n_estim100)",0.718968,0.699250,0.160103,0.712834,0.261478,0.769154
3,GBT,"(learn rate=0.2,max depth=5,n_estim100)",0.777075,0.716333,0.162270,0.672173,0.261428,0.761040
4,GBT,"(learn rate=0.2,max depth=4,n_estim100)",0.751928,0.706463,0.160083,0.689962,0.259871,0.763691
5,GBT,"(learn rate=0.1,max depth=3,n_estim100)",0.711956,0.696403,0.158744,0.712834,0.259662,0.770812
6,GBT,"(learn rate=0.3,max depth=3,n_estim100)",0.734509,0.702857,0.159500,0.697586,0.259636,0.767762
7,GBT,"(learn rate=0.4,max depth=6,n_estim100)",0.866999,0.762836,0.169243,0.556544,0.259556,0.735540
8,Logistic Regression,"(C=0.1,default,na)",0.704159,0.691848,0.158143,0.722999,0.259521,0.771991
9,GBT,"(learn rate=0.1,max depth=6,n_estim100)",0.777791,0.712821,0.160243,0.670902,0.258697,0.765204


In [ ]:
df_model.to_csv(path + "test.csv", index=False)

In [ ]:
#df_modelhold = df_modelhold.sort_values(by = 'Recall', ascending=False).reset_index(drop=True)
#df_modelhold[:20]

### RESULTS OF HOLDOUT USING BEST MODEL

In [10]:
X_train2 = X_train.copy()
y_train2 = y_train.copy()

X_hold2 = df2hold.copy()
X_hold2 = DropFields(X_hold2)
print ("Training shape",X_train2.shape )
#X_hold_raw = X_hold.copy()
#X_hold2 = X_hold.copy()
#X_hold = df_hold.copy()
y_hold2 = X_hold2["Outcome"]
X_hold2 = X_hold2.drop('Outcome', 1)
#print ("Reducing number of features")
#X_hold2 = X_hold2[importantfeatures]
print ("Holdout shape",X_hold2.shape )

#fit the best model
#neural networks
#clf = MLPClassifier(hidden_layer_sizes = ([100, 100, 100]), max_iter=1000) 
#naive bayes
#clf = GaussianNB()
#random forests
#clf = RandomForestClassifier(n_estimators = 30,random_state=0)
#Gradient Boosted Trees
clf = GradientBoostingClassifier(learning_rate = 0.4, max_depth = 2, random_state = 0)
#clf = GradientBoostingClassifier(learning_rate = 0.001, max_depth = 6, random_state = 0)

#logistic regression
#clf = LogisticRegression(C=1)

clf.fit(X_train2, y_train2)

from sklearn.metrics import confusion_matrix
clf_predicted = clf.predict(X_hold2)
confusion = confusion_matrix(y_hold2, clf_predicted)
print ("Confusion Matrix")
print (confusion)


print ("Calculate predictions using best model")
result  =   clf.predict_proba(X_hold2)
#assign positive class probability predictions
    
X_hold2["PredProb"] = result[:,1]

X_hold2 = pd.concat([X_hold2, y_hold2], axis=1)

print ("create predicted outcome")
X_hold2["PredOutcome"] = 0

X_hold2["Index"] = X_hold2.index
X_hold2 = X_hold2.reset_index(drop=True)

for i in range(0, len(X_hold2)):
    if X_hold2["PredProb"][i] >= 0.5:
        X_hold2["PredOutcome"][i] = 1
        
print ("DONE")

print (clf)
print ("Size of final test data (holdout data):", len(X_hold2))
print ("True positive:", sum(confusion[1]) )
print ("Correct Predictions:",confusion[1][1] )
print ("Percent correctly predicted:",round(float(confusion[1][1])/sum(confusion[1]), 2) )
print ("False negatives:", confusion[1][0])
print ("Percent false negatives:",  round(confusion[1][0]/sum(confusion[1]),2) )
print ("False positives:", confusion[0][1] )
print ("Percent false positives:", round(float(confusion[0][1])/len(X_hold2), 2) )

Training shape (58616, 123)
Holdout shape (10537, 123)
Confusion Matrix
[[6893 2844]
 [ 256  544]]
Calculate predictions using best model
create predicted outcome
DONE
GradientBoostingClassifier(learning_rate=0.4, max_depth=2, random_state=0)
Size of final test data (holdout data): 10537
True positive: 800
Correct Predictions: 544
Percent correctly predicted: 0.68
False negatives: 256
Percent false negatives: 0.32
False positives: 2844
Percent false positives: 0.27


In [ ]:
Holdout shape (10537, 123)
Confusion Matrix
[[6798 2946]
 [ 228  565]]
Calculate predictions using best model
create predicted outcome
DONE
GradientBoostingClassifier(learning_rate=0.4, max_depth=2, random_state=0)
Size of final test data (holdout data): 10537
True positive: 793
Correct Predictions: 565
Percent correctly predicted: 0.71
False negatives: 228
Percent false negatives: 0.29
False positives: 2946
Percent false positives: 0.28

### GET FEATURE IMPORTANCE 

In [11]:
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(X_train2.columns, clf.feature_importances_):
    feats[feature] = importance #add the name/value pair 
    #print (feature, importance)
    
x = pd.DataFrame(list(feats.items()),columns = ['Feature','Importance']) 

x = x.sort_values(by = 'Importance', ascending=False).reset_index(drop=True)

#for i in range(0, len(x)):
#    print (x["Feature"][i], x["Importance"][i].round(3))

x2 = x[x["Importance"] > 0.01]

importantfeatures = x2["Feature"].tolist()

In [12]:
x[:20]

,Feature,Importance
0,txt_basement,0.122239
1,txt_fdny,0.100288
2,txt_cellar,0.100145
3,BA_<3000,0.093625
4,CB_412,0.077323
5,HPDNA,0.059873
6,BA_3000_6000,0.052832
7,CB_413,0.047394
8,HighRise,0.023797
9,CB_311,0.017794


In [11]:
print (len(importantfeatures))
importantfeatures

16


['txt_cellar',
 'txt_basement',
 'txt_fdny',
 'BA_<3000',
 'CB_412',
 'BA_3000_6000',
 'HPDNA',
 'CB_413',
 'OT_HeadOfficer',
 'HighRise',
 'txt_occupancy',
 'CB_410',
 'CB_409',
 'MedIncome_High',
 'BA_>20000',
 'CB_311']

In [ ]:
import numpy
def plot_feature_importances(clf, feature_names):
    c_features = len(feature_names)
    plt.barh(range(c_features), clf.feature_importances_)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature name")
    plt.yticks(numpy.arange(c_features), feature_names)
    
plot_feature_importances(clf, feature_names)

### PREDICTIONS USING BEST MODEL

In [15]:
opendf.dtypes[:50]

DateReceived                    datetime64[ns]
ComplaintCategory                        int64
ComplaintCategoryDescription            object
ComplaintNumber                          int64
BIN                                     object
CommunityBoard                          object
BoroughName                             object
SourceofComplaintDescription            object
Latitude                               float64
Longitude                              float64
P_NUMBER_OF_STORIES                      int64
DOFBuildingClassification               object
BBL_SPATIAL_KEY                         object
FinanceOwnerName                        object
HouseNumber                             object
StreetName                              object
RegistrationID                         float64
Outcome                                 object
ComplaintReason                         object
BldgClass                               object
HPD                                    float64
HPDNA        

In [20]:
opendf2 = opendf.copy()
opendf2["Index"] = opendf2.index
opendf2 = DropFields(opendf2)
opendf2 = opendf2.drop('Outcome', 1)

#opendf2 = opendf2[importantfeatures]

print (opendf2.shape)

result  =   clf.predict_proba(opendf2)
#assign positive class probability predictions
    
opendf2["PredProb"] = result[:,1]


#X_test_im = pd.concat([X_test_im, y_test_im], axis=1)

print ("create predicted outcome")
opendf2["PredOutcome"] = 0

#X_test_im["Index"] = X_test_im.index
opendf2 = opendf2.reset_index(drop=True)

for i in range(0, len(opendf2)):
    if opendf2["PredProb"][i] >= 0.5:
        opendf2["PredOutcome"][i] = 1
        
print ("Length of opendf2", len(opendf2))
print ("Number of positive predicted outcomes",opendf2["PredOutcome"].sum() )




(4089, 123)
create predicted outcome
Length of opendf2 4089
Number of positive predicted outcomes 2108


In [21]:
print ("Merge to get categorical data")
opendfcolumns = opendf.columns[:41]

print ("opendf shape", opendf.shape)
print ("opedf2 shape", opendf2.shape)

output = opendf2[["PredProb","PredOutcome"]]
opendfoutput = opendf[opendfcolumns]

print ("output shape", output.shape)
print ("opendfoutput shape", opendfoutput.shape)



output = output.reset_index(drop=True)
opendfoutput = opendfoutput.reset_index(drop=True)

output2  = pd.concat([output, opendfoutput], axis=1)

print ("output2 shape", output2.shape)



Merge to get categorical data
opendf shape (4089, 151)
opedf2 shape (4089, 125)
output shape (4089, 2)
opendfoutput shape (4089, 41)
output2 shape (4089, 43)


In [19]:
opendfcolumns = opendf.columns[:41]
opendfcolumns

Index(['DateReceived', 'ComplaintCategory', 'ComplaintCategoryDescription',
       'ComplaintNumber', 'BIN', 'CommunityBoard', 'BoroughName',
       'SourceofComplaintDescription', 'Latitude', 'Longitude',
       'P_NUMBER_OF_STORIES', 'DOFBuildingClassification', 'BBL_SPATIAL_KEY',
       'FinanceOwnerName', 'HouseNumber', 'StreetName', 'RegistrationID',
       'Outcome', 'ComplaintReason', 'BldgClass', 'HPD', 'HPDNA', 'YearBuilt',
       'BuildingArea', 'Owner', 'OwnerType', 'MD', 'MedianIncome', 'HighRise',
       'YearBuiltCat', 'BuildingAreaCat', 'MedianIncomeCategory',
       'NoPreviousJob', 'PreviousClass1', 'PreviousSale', 'PreviousVacate',
       'txt_fdny', 'txt_occupancy', 'txt_cellar', 'txt_attic', 'txt_basement'],
      dtype='object')

In [22]:
output2.to_csv(path + "scoring on open data 2.csv", index=False)